## Importando bibliotecas

In [3]:
from pyomo.environ import *
from power import *
import numpy as np
import pandas as pd

## DADOS

In [26]:
# Dados dos geradores
dger_data = {
    "PGMIN":     [0, 0, 0, 0],
    "PGMAX":     [455, 455, 130, 130],
    "COST_A":    [1000, 970, 700, 680],
    "COST_B":    [16.19, 17.26, 16.60, 16.50],
    "COST_C":    [0.00048, 0.00031, 0.00200, 0.00211],
    "MTU":       [8, 8, 5, 5],
    "MTD":       [8, 8, 5, 5],
    "HOT_COST":  [4500, 5000, 550, 560],
    "COLD_COST": [9000, 10000, 1100, 1120],
    "HTC":       [5, 5, 4, 4],
}

generators = pd.DataFrame(dger_data)
generators.index.name = 'G'

# Dados da demanda por hora
dload_data = {
    "DEMAND":   [450, 530, 600, 540, 400, 280, 290, 500],
    "SPIN_RES": [45, 53, 60, 54, 40, 28, 29, 50],
}
demand = pd.DataFrame(dload_data)
demand.index = range(1, len(demand) + 1)
demand.index.name = 'T'

# Sets
G = generators.index.tolist()     # Geradores
T = demand.index.tolist()         # Horas


### ISB E ISC

In [28]:
NGER = len(generators)

ISD = np.zeros((NGER, 1))
ISB = np.zeros((NGER, 1))
ISC = np.zeros((NGER, 1))

for ger in range(NGER):
    cost_b = generators["COST_B"].iloc[ger]
    cost_c = generators["COST_C"].iloc[ger]
    pgmin = generators["PGMIN"].iloc[ger]
    pgmax = generators["PGMAX"].iloc[ger]
    mtu = generators["MTU"].iloc[ger]
    hot_cost = generators["HOT_COST"].iloc[ger]
    cold_cost = generators["COLD_COST"].iloc[ger]

    # ISB
    ISB[ger, 0] = cost_b + 2 * cost_c * (pgmax - pgmin) / 2

    # CP
    CP = (hot_cost + cold_cost) / 2

    # ISC
    ISC[ger, 0] = cost_b + 2 * cost_c * pgmax / 2 + CP / (mtu * pgmax)

# Se quiser ver como DataFrame
df_indices = pd.DataFrame({
    "ISB": ISB.flatten(),
    "ISC": ISC.flatten()
}, index=generators.index)

print(df_indices)
print("ISC:", ISC)

        ISB        ISC
G                     
0  16.40840  18.262796
1  17.40105  19.461490
2  16.86000  18.129231
3  16.77430  18.066608
ISC: [[18.2627956 ]
 [19.46148956]
 [18.12923077]
 [18.06660769]]


### ISD

In [12]:
def solve_periodic_dispatch(generators, demand):
    G = generators.index.tolist()
    results = {}

    for t in demand.index:  # Um modelo por período
        model = ConcreteModel()

        # -------------------- Sets
        model.G = Set(initialize=G)  # Apenas conjunto de geradores

        # -------------------- Parâmetros dos geradores
        model.PGMIN     = Param(model.G, initialize=generators["PGMIN"].to_dict())
        model.PGMAX     = Param(model.G, initialize=generators["PGMAX"].to_dict())
        model.COST_A    = Param(model.G, initialize=generators["COST_A"].to_dict())
        model.COST_B    = Param(model.G, initialize=generators["COST_B"].to_dict())
        model.COST_C    = Param(model.G, initialize=generators["COST_C"].to_dict())
        model.MTU       = Param(model.G, initialize=generators["MTU"].to_dict())
        model.MTD       = Param(model.G, initialize=generators["MTD"].to_dict())
        model.HOT_COST  = Param(model.G, initialize=generators["HOT_COST"].to_dict())
        model.COLD_COST = Param(model.G, initialize=generators["COLD_COST"].to_dict())
        model.HTC       = Param(model.G, initialize=generators["HTC"].to_dict())

        # -------------------- Parâmetros do tempo atual
        demand_t = demand.loc[t, "DEMAND"]
        reserve_t = demand.loc[t, "SPIN_RES"]

        # -------------------- Variáveis
        model.PG = Var(model.G, within=NonNegativeReals)

        # -------------------- Função objetivo
        def obj_rule(m):
            return sum(m.COST_A[g] + m.COST_B[g]*m.PG[g] + m.COST_C[g]*m.PG[g]**2 for g in m.G)
        model.cost = Objective(rule=obj_rule, sense=minimize)

        # -------------------- Restrições
        model.balance = Constraint(expr=sum(model.PG[g] for g in model.G) == demand_t)

        model.pgmin = ConstraintList()
        model.pgmax = ConstraintList()
        for g in model.G:
            model.pgmin.add(model.PG[g] >= model.PGMIN[g])
            model.pgmax.add(model.PG[g] <= model.PGMAX[g])


        # -------------------- Solver
        solver = SolverFactory('ipopt')  # ou outro compatível
        solver.solve(model)

        # -------------------- Resultados
        generation = {g: value(model.PG[g]) for g in model.G}
        cost_total = value(model.cost)

        results[t] = {
            "geracao": generation,
            "custo": cost_total,
            "demanda": demand_t,
            "reserva": reserve_t
        }
    return results

In [13]:
# Rodar o despacho para cada período
resultado = solve_periodic_dispatch(generators, demand)

# Inicializar dicionário para construção do DataFrame de gerações
geracao_por_tempo = {}

for t in resultado:
    geracao_por_tempo[t] = resultado[t]["geracao"]

# Converter para DataFrame
geracao_df = pd.DataFrame.from_dict(geracao_por_tempo, orient='index')
geracao_df.index.name = 'Tempo'
geracao_df.columns = [f'G{g+1}' for g in geracao_df.columns]

# Exibir resultados
print("\nGeração por período:")
print(geracao_df.round(2))


Geração por período:
           G1   G2     G3     G4
Tempo                           
1      426.45  0.0   0.00  23.55
2      455.00  0.0  26.34  48.66
3      455.00  0.0  62.27  82.73
4      455.00  0.0  31.47  53.53
5      385.71  0.0   0.00  14.29
6      280.00  0.0   0.00   0.00
7      290.00  0.0   0.00   0.00
8      455.00  0.0  10.94  34.06


In [15]:
# Inicializa DataFrame do ISD com mesmo shape do de geração
isd_df = pd.DataFrame(index=geracao_df.index, columns=geracao_df.columns)

# Loop sobre os geradores e tempos
for g in generators.index:
    a = generators.loc[g, 'COST_A']
    b = generators.loc[g, 'COST_B']
    c = generators.loc[g, 'COST_C']
    g_col = f'G{g+1}'  # Nome da coluna no DataFrame de geração

    for t in geracao_df.index:
        pg = geracao_df.loc[t, g_col]
        if pg > 0:
            isd = (a + b * pg + c + c * pg**2) / pg
        else:
            isd = np.nan  # ou isd = 0, se preferir

        isd_df.loc[t, g_col] = isd

# Converter para float
isd_df = isd_df.astype(float)

# Exibir resultados arredondados
print("\nISD por período:")
print(isd_df.round(2))


ISD por período:
          G1            G2            G3            G4
Tempo                                                 
1      18.74           NaN  8.474829e+07  4.542000e+01
2      18.61           NaN  4.323000e+01  3.058000e+01
3      18.61  4.413514e+11  2.797000e+01  2.489000e+01
4      18.61           NaN  3.890000e+01  2.932000e+01
5      18.97           NaN  5.948522e+09  6.413000e+01
6      19.90           NaN  2.740662e+10  6.012674e+09
7      19.78           NaN  2.483641e+10  4.127320e+09
8      18.61           NaN  8.063000e+01  3.653000e+01


In [ ]:
for i in range(len(generators)):
    

0
1
2
3
